In [1]:
import sys
import json
import csv
import yaml

import importlib

import math

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

from datetime import time
from datetime import date
from datetime import datetime
# with the above choices, the imported datetime.time(2023,07,01) is recognized
# from datetime import date
# from datetime import datetime

import pprint

import psycopg2
from sqlalchemy import create_engine, text as sql_text

# Create an utilities file util.py in a folder benchmarking and import it
# NOTE: I moved my util.py to the directory "helper_functions" -- seems like a better name
# sys.path.append('helper_functions/')
# import util

In [2]:
from pymongo import MongoClient
from bson import ObjectId

client = MongoClient()
# could have written client = MongoClient("localhost", 27017)
#                 or client = MongoClient("mongodb://localhost:27017/")

In [3]:
# I have (or will have) a database "airbnb"
db = client.exam_questions


In [4]:
val1 = { 'employer_name': 'UC Davis',
         'employer_city': 'Davis',
         'industry': 'University' }
val2 = { 'employer_name': 'UC Berkeley',
         'employer_city': 'Berkeley',
         'industry': 'University' }
val3 = { 'employer_name': 'UC Santa Barbara',
         'employer_city': 'Santa Barbara',
         'industry': 'University' }

db.employer.drop()
db.employer.insert_many([val1, val2, val3])
cursor = db.employer.find()
for doc in cursor:
    pprint.pp(doc)

{'_id': ObjectId('66675dea154d7d6907a930b5'),
 'employer_name': 'UC Davis',
 'employer_city': 'Davis',
 'industry': 'University'}
{'_id': ObjectId('66675dea154d7d6907a930b6'),
 'employer_name': 'UC Berkeley',
 'employer_city': 'Berkeley',
 'industry': 'University'}
{'_id': ObjectId('66675dea154d7d6907a930b7'),
 'employer_name': 'UC Santa Barbara',
 'employer_city': 'Santa Barbara',
 'industry': 'University'}


In [5]:
ucd_id = db.employer.find_one({'employer_name':'UC Davis'})['_id']
ucb_id = db.employer.find_one({'employer_name':'UC Berkeley'})['_id']
ucsb_id = db.employer.find_one({'employer_name':'UC Santa Barbara'})['_id']





val4 = { 'name': 'John Smith',
        'region': 'Chicago',
        'summary': 'Hard-working Data Scientist',
        'work_history': [ { 'employer__id': ucd_id,
                            'start_date': '2000-01-01',
                            'end_date': '2010-01-01' },
                          { 'employer__id': ucsb_id,
                            'start_date': '2011-01-01'}
                        ]
      }
val5 = { 'name': 'Jane Doe',
        'region': 'Chicago',
        'summary': 'Hard-working Software Engineer',
        'work_history': [ {'employer__id': ucsb_id,
                           'start_date': '2000-01-01',
                           'end_date': '2010-01-01' },
                          {'employer__id': ucb_id,
                           'start_date': '2011-01-01'}
                        ]
      }
                           
val6 = { 'name': 'Janet Simpson',
        'region': 'Chicago',
        'summary': 'Hard-working Electrical Engineer',
        'work_history': [ {'employer__id': ucsb_id,
                           'start_date': '2000-01-01',
                           'end_date': '2002-01-01' },
                          {'employer__id': ucb_id,
                           'start_date': '2015-01-01',
                           'end_date': '2017_01_01'},
                          {'employer__id': ucd_id,
                           'start_date': '2019-01-01'}
                        ]
      }

db.person.drop()
db.person.insert_many([val4, val5, val6])
cursor = db.person.find()
for doc in cursor:
    pprint.pp(doc)

{'_id': ObjectId('66675dea154d7d6907a930b8'),
 'name': 'John Smith',
 'region': 'Chicago',
 'summary': 'Hard-working Data Scientist',
 'work_history': [{'employer__id': ObjectId('66675dea154d7d6907a930b5'),
                   'start_date': '2000-01-01',
                   'end_date': '2010-01-01'},
                  {'employer__id': ObjectId('66675dea154d7d6907a930b7'),
                   'start_date': '2011-01-01'}]}
{'_id': ObjectId('66675dea154d7d6907a930b9'),
 'name': 'Jane Doe',
 'region': 'Chicago',
 'summary': 'Hard-working Software Engineer',
 'work_history': [{'employer__id': ObjectId('66675dea154d7d6907a930b7'),
                   'start_date': '2000-01-01',
                   'end_date': '2010-01-01'},
                  {'employer__id': ObjectId('66675dea154d7d6907a930b6'),
                   'start_date': '2011-01-01'}]}
{'_id': ObjectId('66675dea154d7d6907a930ba'),
 'name': 'Janet Simpson',
 'region': 'Chicago',
 'summary': 'Hard-working Electrical Engineer',
 'work_histor

In [6]:
ucd = db.employer.find_one({'employer_name':'UC Davis'})
print(ucd)
print(type(ucd))

{'_id': ObjectId('66675dea154d7d6907a930b5'), 'employer_name': 'UC Davis', 'employer_city': 'Davis', 'industry': 'University'}
<class 'dict'>


In [7]:
ucd_id = ucd['_id']
print(ucd_id)

66675dea154d7d6907a930b5


In [8]:
print(db.person.count_documents({}))
print(db.employer.count_documents({}))

3
3


In [9]:
cursor = db.person.find({'work_history.employer__id': ucd_id})
for doc in cursor:
    pprint.pp(doc)

{'_id': ObjectId('66675dea154d7d6907a930b8'),
 'name': 'John Smith',
 'region': 'Chicago',
 'summary': 'Hard-working Data Scientist',
 'work_history': [{'employer__id': ObjectId('66675dea154d7d6907a930b5'),
                   'start_date': '2000-01-01',
                   'end_date': '2010-01-01'},
                  {'employer__id': ObjectId('66675dea154d7d6907a930b7'),
                   'start_date': '2011-01-01'}]}
{'_id': ObjectId('66675dea154d7d6907a930ba'),
 'name': 'Janet Simpson',
 'region': 'Chicago',
 'summary': 'Hard-working Electrical Engineer',
 'work_history': [{'employer__id': ObjectId('66675dea154d7d6907a930b7'),
                   'start_date': '2000-01-01',
                   'end_date': '2002-01-01'},
                  {'employer__id': ObjectId('66675dea154d7d6907a930b6'),
                   'start_date': '2015-01-01',
                   'end_date': '2017_01_01'},
                  {'employer__id': ObjectId('66675dea154d7d6907a930b5'),
                   'start_date'

In [10]:
# db.test1.drop()
pipeline = [ { '$match' : { 'work_history.employer__id': ucd_id}},
             { '$unset': ['_id', 'work_history'] },
             # { '$out' : 'test1' }
           ]

cursor = db.person.aggregate(pipeline)

for doc in cursor:
    pprint.pp(doc)

{'name': 'John Smith',
 'region': 'Chicago',
 'summary': 'Hard-working Data Scientist'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'summary': 'Hard-working Electrical Engineer'}


In [11]:
db.test1.drop()

pipeline = [ # 
            { '$unset' : '_id' },
            { '$unwind' : '$work_history'},
            { '$match' : { 'work_history._id': ucd_id}},
            { '$project': {
                   'name' : '$name',
                   'region' : '$region',
                   'start_date' : '$work_history.start_date',
                   'end_date' : '$work_history.end_date'
                   }
            },
             { '$out' : 'test1' }
           ]

db.person.aggregate(pipeline)

ans = db.test1.find({})
for doc in ans:
    pprint.pp(doc)

### <span style=color:blue>Creating records of shape person_name, region, employer_name, start_date, end_date -- have to to an unwind, a project (to flatten the work history with employer ObjectID), then a lookup (join) to bring in data from Employer, and then drop some columns   </span>

In [12]:
db.test2.drop()

pipeline = [  { '$unset' : '_id' },
              { '$unwind' : '$work_history'},
              { '$project': {
                   'name' : '$name',
                   'region' : '$region',
                   'employer__id' : '$work_history.employer__id',
                   'start_date' : '$work_history.start_date',
                   'end_date' : '$work_history.end_date'
              }}
          ]
"""
              { '$lookup' : {
                   'from': 'employer',
                   'localField':  'employer__id',
                   'foreignField': '_id',
                   'as' : 'employer_list'
              }},
              { '$unwind' : '$employer_list'},
              { '$addFields' : { 'employer_name' : '$$ROOT.employer_list.employer_name'}},    
              { '$unset' : ['employer_list', 'employer__id'] },
              { '$out' : 'test2' }
          ]
"""


cursor = db.person.aggregate(pipeline)

for doc in cursor:
    pprint.pp(doc)


{'name': 'John Smith',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b5'),
 'start_date': '2000-01-01',
 'end_date': '2010-01-01'}
{'name': 'John Smith',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b7'),
 'start_date': '2011-01-01'}
{'name': 'Jane Doe',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b7'),
 'start_date': '2000-01-01',
 'end_date': '2010-01-01'}
{'name': 'Jane Doe',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b6'),
 'start_date': '2011-01-01'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b7'),
 'start_date': '2000-01-01',
 'end_date': '2002-01-01'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b6'),
 'start_date': '2015-01-01',
 'end_date': '2017_01_01'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b5'),
 'start_date'

In [13]:
db.test2.drop()

pipeline = [  { '$unset' : '_id' },
              { '$unwind' : '$work_history'},
              { '$project': {
                   'name' : '$name',
                   'region' : '$region',
                   'employer__id' : '$work_history.employer__id',
                   'start_date' : '$work_history.start_date',
                   'end_date' : '$work_history.end_date'
              }},
              { '$lookup' : {
                   'from': 'employer',
                   'localField':  'employer__id',
                   'foreignField': '_id',
                   'as' : 'employer_list'
              }},
          ]
"""
              { '$unwind' : '$employer_list'},
              { '$addFields' : { 'employer_name' : '$$ROOT.employer_list.employer_name'}},    
              { '$unset' : ['employer_list', 'employer__id'] },
              { '$out' : 'test2' }
          ]
"""


cursor = db.person.aggregate(pipeline)

for doc in cursor:
    pprint.pp(doc)


{'name': 'John Smith',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b5'),
 'start_date': '2000-01-01',
 'end_date': '2010-01-01',
 'employer_list': [{'_id': ObjectId('66675dea154d7d6907a930b5'),
                    'employer_name': 'UC Davis',
                    'employer_city': 'Davis',
                    'industry': 'University'}]}
{'name': 'John Smith',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b7'),
 'start_date': '2011-01-01',
 'employer_list': [{'_id': ObjectId('66675dea154d7d6907a930b7'),
                    'employer_name': 'UC Santa Barbara',
                    'employer_city': 'Santa Barbara',
                    'industry': 'University'}]}
{'name': 'Jane Doe',
 'region': 'Chicago',
 'employer__id': ObjectId('66675dea154d7d6907a930b7'),
 'start_date': '2000-01-01',
 'end_date': '2010-01-01',
 'employer_list': [{'_id': ObjectId('66675dea154d7d6907a930b7'),
                    'employer_name': 'UC Santa Barbara',
       

In [15]:
pipeline = [  { '$unset' : '_id' },
              { '$unwind' : '$work_history'},
              { '$project': {
                   'name' : '$name',
                   'region' : '$region',
                   'employer__id' : '$work_history.employer__id',
               #    'employer_name': '$work_history.employer_name',
                   'start_date' : '$work_history.start_date',
                   'end_date' : '$work_history.end_date'
              }},
              { '$lookup' : {
                   'from': 'employer',
                   'localField':  'employer__id',
                   'foreignField': '_id',
                   'as' : 'employer_list'
              }},
              { '$unwind' : '$employer_list'},
              { '$addFields' : { 'employer_name' : '$$ROOT.employer_list.employer_name'}},
              { '$unset' : ['employer_list', 'employer__id'] }
          ]

cursor = db.person.aggregate(pipeline)

for doc in cursor:
    pprint.pp(doc)


{'name': 'John Smith',
 'region': 'Chicago',
 'start_date': '2000-01-01',
 'end_date': '2010-01-01',
 'employer_name': 'UC Davis'}
{'name': 'John Smith',
 'region': 'Chicago',
 'start_date': '2011-01-01',
 'employer_name': 'UC Santa Barbara'}
{'name': 'Jane Doe',
 'region': 'Chicago',
 'start_date': '2000-01-01',
 'end_date': '2010-01-01',
 'employer_name': 'UC Santa Barbara'}
{'name': 'Jane Doe',
 'region': 'Chicago',
 'start_date': '2011-01-01',
 'employer_name': 'UC Berkeley'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'start_date': '2000-01-01',
 'end_date': '2002-01-01',
 'employer_name': 'UC Santa Barbara'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'start_date': '2015-01-01',
 'end_date': '2017_01_01',
 'employer_name': 'UC Berkeley'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'start_date': '2019-01-01',
 'employer_name': 'UC Davis'}


In [39]:
pipeline = [
    { '$unwind' : '$work_history' },
    { '$match' : { 'work_history.end_date' : { '$exists' : False }
                 }
    },
    { '$lookup' : 
          { 'from' : 'employer',
            'localField' : 'work_history.employer__id',
            'foreignField' : '_id',
            'as' : 'employer'
          }
    },
    { '$unwind' : '$employer' },
    { '$addFields' : { 'employer_name' : '$$ROOT.employer.employer_name',
                       'employer_city' : '$$ROOT.employer.employer_city',
                       'start_date' : '$$ROOT.work_history.start_date'
                     }
    },
    { '$unset': [ '_id', 'work_history', 'employer' ] 
    }
] 

cursor = db.person.aggregate(pipeline)

for doc in cursor:
    pprint.pp(doc)

{'name': 'John Smith',
 'region': 'Chicago',
 'summary': 'Hard-working Data Scientist',
 'employer_name': 'UC Santa Barbara',
 'employer_city': 'Santa Barbara',
 'start_date': '2011-01-01'}
{'name': 'Jane Doe',
 'region': 'Chicago',
 'summary': 'Hard-working Software Engineer',
 'employer_name': 'UC Berkeley',
 'employer_city': 'Berkeley',
 'start_date': '2011-01-01'}
{'name': 'Janet Simpson',
 'region': 'Chicago',
 'summary': 'Hard-working Electrical Engineer',
 'employer_name': 'UC Davis',
 'employer_city': 'Davis',
 'start_date': '2019-01-01'}
